In [1]:
#模型推理
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.33it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [5]:
#使用原始 ChatGLM3-6B 模型
input_text = "解释下古希腊哲学是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

古希腊哲学是指古希腊时期（公元前5世纪到公元前3世纪）哲学家们对自然、社会、人类和思维等方面的探讨和思考。古希腊哲学的起源可以追溯到公元前6世纪，当时的苏格拉底、柏拉图和亚里士多德等哲学家开创了不同的哲学体系。在随后的时期，哲学家们对同一问题进行了更深入的探讨，发展出了不同的学说。

古希腊哲学的主要内容包括伦理学、形而上学、认识论、逻辑学、政治学等方面。其中，伦理学主要关注道德和道德价值观念的探讨；形而上学关注宇宙、自然和人类本性的研究；认识论探讨知识的本质、来源和范围；逻辑学关注推理和论证的规律；政治学则关注国家、政治制度和社会组织的理论基础。

古希腊哲学的著名学派和哲学家包括：

1. 苏格拉底（Socrates）：被认为是西方哲学的奠基人，他强调个人品德的完善和对知识的追求。

2. 柏拉图（Plato）：创立了形而上学中的理念论，认为真正的现实是理念世界，人类世界只是理念世界的影子。

3. 亚里士多德（Aristotle）：他继承了苏格拉底的理论，对伦理学、形而上学、逻辑学、政治学等多个领域做出了重要贡献。

古希腊哲学对后世产生了深远的影响，它为西方哲学的发展奠定了基础，并影响了欧洲的文学、艺术、科学、政治等领域。同时，古希腊哲学中的许多思想和观念，如民主、平等、逻辑学等，对现代社会产生了重要影响。


In [6]:
response, history = base_model.chat(tokenizer, query="西方哲学史的第一位哲学家是谁？", history=history)
print(response)

西方哲学史通常认为苏格拉底（Socrates）是西方哲学史的第一位哲学家。苏格拉底是古希腊哲学的奠基人，他在公元前5世纪后期和公元前4世纪早期活动于雅典，通过提问和对话的方式引导人们追求智慧和美德。他的方法强调质疑、对话和自我改进，对后世哲学产生了深远影响。虽然有学者认为更早的哲学家如赫拉克利特、巴门尼德等也应该被计入西方哲学史，但通常认为苏格拉底是西方哲学史的开端。


In [7]:
history

[{'role': 'user', 'content': '解释下古希腊哲学是什么？'},
 {'role': 'assistant',
  'metadata': '',
  'content': '古希腊哲学是指古希腊时期（公元前5世纪到公元前3世纪）哲学家们对自然、社会、人类和思维等方面的探讨和思考。古希腊哲学的起源可以追溯到公元前6世纪，当时的苏格拉底、柏拉图和亚里士多德等哲学家开创了不同的哲学体系。在随后的时期，哲学家们对同一问题进行了更深入的探讨，发展出了不同的学说。\n\n古希腊哲学的主要内容包括伦理学、形而上学、认识论、逻辑学、政治学等方面。其中，伦理学主要关注道德和道德价值观念的探讨；形而上学关注宇宙、自然和人类本性的研究；认识论探讨知识的本质、来源和范围；逻辑学关注推理和论证的规律；政治学则关注国家、政治制度和社会组织的理论基础。\n\n古希腊哲学的著名学派和哲学家包括：\n\n1. 苏格拉底（Socrates）：被认为是西方哲学的奠基人，他强调个人品德的完善和对知识的追求。\n\n2. 柏拉图（Plato）：创立了形而上学中的理念论，认为真正的现实是理念世界，人类世界只是理念世界的影子。\n\n3. 亚里士多德（Aristotle）：他继承了苏格拉底的理论，对伦理学、形而上学、逻辑学、政治学等多个领域做出了重要贡献。\n\n古希腊哲学对后世产生了深远的影响，它为西方哲学的发展奠定了基础，并影响了欧洲的文学、艺术、科学、政治等领域。同时，古希腊哲学中的许多思想和观念，如民主、平等、逻辑学等，对现代社会产生了重要影响。'},
 {'role': 'user', 'content': '西方哲学史的第一位哲学家是谁？'},
 {'role': 'assistant',
  'metadata': '',
  'content': '西方哲学史通常认为苏格拉底（Socrates）是西方哲学史的第一位哲学家。苏格拉底是古希腊哲学的奠基人，他在公元前5世纪后期和公元前4世纪早期活动于雅典，通过提问和对话的方式引导人们追求智慧和美德。他的方法强调质疑、对话和自我改进，对后世哲学产生了深远影响。虽然有学者认为更早的哲学家如赫拉克利特、巴门尼德等也应该被计入西方哲学史，但通常认为苏格拉底是西方哲学史的开端。'}]

In [8]:
#使用微调后的 ChatGLM3-6B
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20250824_095254"

peft_model_path = f"/GL8616/models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [9]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [10]:
base_response, ft_response = compare_chatglm_results("解释下古希腊哲学是什么？", base_model, qlora_model, training_tag)

问题：解释下古希腊哲学是什么？

原始输出：
古希腊哲学是指古希腊时期哲学家们的思想和理论,涵盖了伦理学、形而上学、逻辑、政治、美学和科学等多个领域。

古希腊哲学的起源可以追溯到公元前6世纪,当时的古希腊城邦开始出现最早的哲学思考。公元前5世纪,著名的哲学家如苏格拉底、柏拉图和亚里士多德等人开始发展自己的哲学体系。

古希腊哲学中的一些重要思想包括:

1. 原子论:认为所有物质都是由微小的不可分割的粒子——原子组成的。

2. 民主:强调公民的自由和平等,认为政治权力应该由所有公民共同分享。

3. 逻辑学:研究推理和论证的基本原则,包括命题逻辑和谓词逻辑。

4. 伦理学:研究道德和道德价值观念,强调人的道德责任和个人品德的重要性。

5. 形而上学:研究现实和存在的本质,探讨宇宙的起源和结构。

6. 科学:对自然界进行研究和探索,包括物理学、生物学、数学和天文学等。

古希腊哲学对西方哲学和西方文化的发展产生了深远的影响,它的一些思想和观念仍然影响着我们的思考和生活。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250824_095254）：
[gMASK]sop 解释下古希腊哲学是什么？
古希腊哲学是指古希腊时期哲学家们探索人类本质、社会、宇宙等问题的哲学思想。古希腊哲学的起源可以追溯到公元前6世纪,当时的希腊城邦开始出现独立思考的个人,并开始对传统的神话和宗教产生怀疑。

古希腊哲学的主要流派包括柏拉图主义、亚里士多德主义和斯多葛主义等。其中,柏拉图主义主张哲学家统治,认为人类的本性是理性的,而亚里士多德主义则主张寻求真理和智慧,强调实证和经验,斯多葛主义则强调道德和伦理。

古希腊哲学对西方哲学的发展产生了深远的影响,其中许多哲学思想和概念已经成为西方哲学的核心。例如,亚里士多德的伦理学、政治学、逻辑学等对西方哲学的影响至今仍然存在。同时,古希腊哲学中的理性、实证、道德等概念也成为了现代科学和伦理学的基础。


In [11]:
base_response, ft_response = compare_chatglm_results("古代中国哲学？", base_model, qlora_model, training_tag)

问题：古代中国哲学？

原始输出：
古代中国哲学，又称为中国哲学史，是指在古代中国这一历史时期涌现出的哲学思想和理论。中国哲学源远流长，可追溯到公元前2000多年前的黄帝时代。在此，我们主要介绍两个重要的哲学流派：儒家和道家。

儒家是我国古代最具影响力的哲学流派，创立于春秋时期的鲁国。儒家的代表人物是孔子，他的思想体系被称为儒家思想。儒家主张“仁”为宇宙的最高道德原则，强调个人修养、家庭伦理和社会秩序。儒家经典《论语》、《孟子》和《大学》等都是儒家哲学的重要文献。

道家成立于春秋战国时期，创立者为老子。道家主张“道”为宇宙的本源，强调顺应自然、无为而治。道家的代表作品是《道德经》，这部著作阐述了道家的哲学观念和人生哲学。道家哲学对后世产生了深远的影响，如庄子、列子等道家学者对儒家的反思和批判，以及唐宋时期道教的繁荣等。

除了儒家和道家之外，古代中国还有许多其他哲学流派，如法家、墨家、名家等。这些流派各有特点，相互竞争，共同推动了中国哲学的发展。

古代中国哲学对后世产生了深远的影响，对于今天的国人，了解古代中国哲学有助于更好地认识中华文明的博大精深，并为构建现代和谐社会提供有益的启示。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250824_095254）：
[gMASK]sop 古代中国哲学？ 
古代中国哲学，又称为中国哲学史，是指起源于中国并在历史上发展演变的一系列哲学思想和理论。古代中国哲学有着悠久的历史，可以追溯到先秦时期。在漫长的历史过程中，涌现出了许多杰出的哲学家，他们的思想成果对后世产生了深远的影响。

古代中国哲学的主要内容包括儒家、道家、墨家、法家等学派。其中，儒家是古代中国哲学的主流，其创立者是孔子，后来经过孟子的发展，形成了儒家思想体系。儒家主张仁、义、礼、智、信等道德观念，强调个人修养和社会责任，对古代中国的教育、政治、伦理等方面产生了重要影响。

道家哲学起源于先秦时期的道家，其创立者是老子。道家主张道法自然，提倡无为而治，强调内心的宁静和自由。道家的思想对后世产生了很大的影响，尤其是在唐朝时期，道家的思想与佛教相结合，形成了具有中国特色的道教。

墨家是古代中国的一个墨子学派，墨子主张兼爱、非攻、节用等思想。墨子的墨家思想对后世的民间道德观念产生了积极的影响。

法家